In [4]:
import os

DATA_DIR = "data" # This may need to be changed on different machines

# Make sure we're in the correct directory and make sure the data directory exists
if not os.path.exists(DATA_DIR):
    os.chdir("../..") # Move up two directories because we're in src/nb and the data directory/path should be in/start at the root directory 
    assert os.path.exists(DATA_DIR), f"ERROR: DATA_DIR={DATA_DIR} not found"  # If we still can't see the data directory something is wrong


from torch.utils.data import DataLoader

import pandas as pd
from src.lib.decoder_dataset import DecoderDataset


In [6]:
df = pd.read_csv(os.path.join(DATA_DIR, "decoded_cds", "balanced", "dev.csv"), index_col=0)
df = df[:2048] # make this next part go faster
dataset = DecoderDataset(df)

  0%|          | 0/32 [00:00<?, ?it/s]

Some weights of the model checkpoint at models/gpt2_large were not used when initializing GPT2LMHeadModel: ['transformer.extra_embedding_project.bias', 'transformer.extra_embedding_project.weight']
- This IS expected if you are initializing GPT2LMHeadModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing GPT2LMHeadModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/home/bill/spring2022/685/CS685Project/env/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:2279: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(


  0%|          | 0/32 [00:00<?, ?it/s]

In [7]:
print(len(dataset))

2048


In [8]:

data_loader = DataLoader(dataset, batch_size=20, shuffle=True)

for batch in data_loader:
    (style_embed, para_token_embed, para_attn_mask), text_token_ids = batch
    print(style_embed.shape)
    print(para_token_embed.shape)
    print(para_attn_mask.shape)
    print(text_token_ids.shape)
    break
    

torch.Size([20, 768])
torch.Size([20, 51, 1280])
torch.Size([20, 51])
torch.Size([20, 108])
